# Targeted Analysis
See the [Targeted_Analysis.md](https://github.com/biorack/metatlas/blob/main/docs/Targeted_Analysis.md) file on GitHub for documentation on how to use this notebook.

#### Parameters
The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# pylint: disable=invalid-name,missing-module-docstring

# The name of a workflow defined in the configuration file
workflow_name = None

# The name of an analysis within the workflow
analysis_name = None

# source atlas name
source_atlas = None

# if copy_atlas is True, then generate an atlas specifically for this analysis_number
# should only be set to False if an analysis will not be modifying the atlas or RT ranges
copy_atlas = None

# one of 'positive' or 'negative'
polarity = None

# an integer, increment if you need to redo your analysis
# will be appended to your username to create analysis_id
analysis_number = None

# experiment ID that must match the parent folder containing the LCMS output files
# An example experiment ID is '20201116_JGI-AK_LH_506489_SoilWarm_final_QE-HF_HILICZ_USHXG01530'
experiment = None

# group will only be used if their name has a substring match to this list of strings
include_groups = None

# Exclude groups with names containing any of the substrings in this list.
# Generally you will want to include polarities you are not using
# such as ['NEG', 'FPS'] for a positive polarity analysis.
exclude_groups = None

# list of substrings that will group together when creating groups
# this provides additional grouping beyond the default grouping on field #12
groups_controlled_vocab = None

# Exclude files with names containing any of the substrings in this list. Eg., ['peas', 'beans']
exclude_files = None

# Create outputs used to QC the run
generate_qc_outputs = None

# thresholds for filtering out compounds with weak MS1 signals
# set to None to disable a filter
num_points = None
peak_height = None

# threshold for filtering out compounds with poor MS2 spectra similaritiy
# Should be a value in range 0 to 1. Set to None to disable this filter.
msms_score = None

# if True, the post_annotation() function will remove atlas rows marked
# 'Remove' before generating output files
filter_removed = None

# list of tuples contain string with color name and substring pattern.
# Lines in the EIC plot will be colored by the first substring pattern
# that has a match within the name of the hdf5_file. The order they are
# listed in your list is the order they are displayed in the overlays
# (first is front, last is back). Named colors available in matplotlib
# are here: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
# or use hexadecimal values '#000000'. Lines default to black.
line_colors = None

# Set to False to disable check that all compounds have either been
# removed or rated within the annotation GUI before generating outputs.
require_all_evaluated = None

# If True, then create the main set of outputs
generate_analysis_outputs = None

# Groups to be excluded when generating the post annotation outputs:
exclude_groups_for_analysis_outputs = None

# include MSMS fragment ions in the output documents?
# has no effect if generate_post_annotation_outputs is False
export_msms_fragment_ions = None

# Setting this to True will remove the cache of MSMS hits
# if you don't see MSMS data for any of your compounds in RT adjuster GUI,
# then you might want to try settings this to True. However, it will
# make your notebook take significantly longer to run.
# The cache is per experiment, so clearing the cache will impact other
# notebooks for this same experiment.
clear_cache = None

# This value will always be automatically passed in from the RT_Alignment
# notebook and you should not manually set this parameter.
rt_alignment_number = None

# The rest of this block contains project independent parameters

# Configuration file location
config_file_name = None

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# Full path to the directory where you want this notebook to store data.
# A subdirectory will be auto created within this directory for each project.
# You can place this anywhere on cori's filesystem, but placing it within your
# global home directory is recommended so that you do not need to worry about
# your data being purged. Each project will take on the order of 100 MB.
project_directory = None

# ID from Google Drive URL for base output folder .
# The default value is the ID that corresponds to 'JGI_Metabolomics_Projects'.
google_folder = None

# maximum number of CPUs to use
# when running on jupyter.nersc.gov, you are not allowed to set this above 4
max_cpus = None

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = None

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import logging  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


parameters = {k: v for k, v in globals().items() if k[0] != "_" and k not in ["In", "Out", "get_ipython", "exit", "quit"]}
logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=wjhjgi/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from metatlas.tools import config, notebook  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err
configuration, workflow, analysis = config.get_config(parameters)
notebook.setup(analysis.parameters.log_level, analysis.parameters.source_code_version_id)
from metatlas.targeted.process import pre_annotation, annotation_gui, post_annotation  # noqa: E402

In [ ]:
metatlas_dataset = pre_annotation(
    experiment=experiment,
    rt_alignment_number=rt_alignment_number,
    analysis_number=analysis_number,
    source_atlas=source_atlas,
    configuration=configuration,
    workflow=workflow,
    analysis=analysis,
    clear_cache=clear_cache,
)

#### Annotation GUI
If you are re-running this notebook and do not need to make additional changes to RT min/max bounds, then you can skip running the next code cell. Skipping will save you from calculating MSMS hits twice.

In [ ]:
agui = annotation_gui(data=metatlas_dataset, compound_idx=0, width=15, height=3, colors=analysis.parameters.line_colors)

In [ ]:
post_annotation(data=metatlas_dataset, configuration=configuration, workflow=workflow, analysis=analysis)